In [1]:
# BILIOTECAS
import pandas as pd
from urllib.request import Request, urlopen
import plotly.express as px
import plotly.io as pio # RENDER IN NOTEBOOK
#import chart_studio
#import chart_studio.plotly as py

# IMPORTANDO API-KEY CHART STUDIO
#from covid.api_key import api_key

In [2]:
#pio.renderers.default = "notebook_connected" # PARA RENDERIZAR NO NOTEBOOK
#chart_studio.tools.set_credentials_file(username = 'user.name', api_key = api_key) # AUTH PARA CHART STUDIO

In [3]:
# DATA 1: COVID
req = Request('https://data.brasil.io/dataset/covid19/caso_full.csv.gz')
req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
content = urlopen(req)
df_covid = pd.read_csv(content, compression='gzip')

In [4]:
# DATA 2: LATLONG
df_latlong = pd.read_csv('https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv')

In [5]:
# treatment and cleaning
df_covid = df_covid[['city','city_ibge_code','epidemiological_week','new_confirmed','new_deaths']]
df_covid = df_covid.loc[df_covid['city_ibge_code'] > 100] # Excluir Estados
df_covid = df_covid.loc[pd.notna(df_covid['city_ibge_code'])]
df_covid['city_ibge_code'] = df_covid['city_ibge_code'].astype('int64')
df_covid = df_covid.loc[df_covid['new_confirmed'] >= 0] 

In [6]:
# join df_covid and df_latlong
df_latlong.set_index('codigo_ibge')
df = df_covid.join(df_latlong.set_index('codigo_ibge'), on = 'city_ibge_code', how = 'left')

df = df[['city','city_ibge_code','epidemiological_week','new_confirmed','latitude','longitude']]

In [7]:
# group by epidemiological_week
df_final = df.groupby(['city','city_ibge_code','epidemiological_week','latitude','longitude'], as_index = False).sum()

# final adjust
df_final = df_final.sort_values(by = ['epidemiological_week'], ascending = True)

In [18]:
fig = px.scatter_mapbox(df_final,
                        title = 'Evolução de casos notificados de COVID no Brasil a partir da 09º Semana Epidemiológica de 2020',
                        lat = 'latitude', 
                        lon = 'longitude', 
                        size = 'new_confirmed', 
                        mapbox_style='carto-positron',
                        zoom = 3.5, 
                        animation_frame = 'epidemiological_week',
                        hover_name = 'city',
                        color_discrete_sequence=["red"],
                        size_max = 20,
                        opacity = 0.5,
                        )

In [ ]:
fig.write_html("covid/file.html")

In [ ]:
#py.plot(fig, filename = 'plotly_mapbox_scatter', auto_open=True) # PARA SUBIR NA CLOUD CHART_STUDIO
#fig.show() # PARA RENDERIZAR NO NOTEBOOK